Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-187534
aml-quickstarts-187534
southcentralus
5a4ab2ba-6c51-4805-8155-58759ad589d8


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-bike-experiment-1'
project_folder = './pipeline-bike-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-bike-experiment-1,quick-starts-ws-187534,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
#amlcompute_cluster_name = "auto-ml"
amlcompute_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [7]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Bikesharing Dataset"
description_text = "Bike Sharing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Azure/MachineLearningNotebooks/master/how-to-use-azureml/automated-machine-learning/forecasting-bike-share/bike-no.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
#df.describe()

### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [8]:
dataset.take(5).to_pandas_dataframe()

,instant,date,season,yr,mnth,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,6,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,2,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,3,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [9]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'normalized_root_mean_squared_error',
    "n_cross_validations": 3
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "forecasting",
                             training_data=dataset,
                             time_column_name="date", 
                             label_column_name="cnt",  
                             path = project_folder,
                             enable_early_stopping= True,
                             #featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [10]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [11]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [12]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [13]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [e92ca11c][3543584e-1a16-4270-bfa8-d262c74bbb91], (This step will run and generate new outputs)
Submitted PipelineRun 5dd4f320-7a3a-45a5-b874-3b1d403bd3ae
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/5dd4f320-7a3a-45a5-b874-3b1d403bd3ae?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-187534/workspaces/quick-starts-ws-187534&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [14]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [15]:
pipeline_run.wait_for_completion()

PipelineRunId: 5dd4f320-7a3a-45a5-b874-3b1d403bd3ae
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/5dd4f320-7a3a-45a5-b874-3b1d403bd3ae?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-187534/workspaces/quick-starts-ws-187534&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: 4d85652b-e60d-4510-abed-8246d05fa168
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/4d85652b-e60d-4510-abed-8246d05fa168?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-187534/workspaces/quick-starts-ws-187534&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/py

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [16]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/4d85652b-e60d-4510-abed-8246d05fa168/metrics_data, 1 files out of an estimated total of 1


In [17]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,4d85652b-e60d-4510-abed-8246d05fa168_13,4d85652b-e60d-4510-abed-8246d05fa168_10,4d85652b-e60d-4510-abed-8246d05fa168_3,4d85652b-e60d-4510-abed-8246d05fa168_6,4d85652b-e60d-4510-abed-8246d05fa168_2,4d85652b-e60d-4510-abed-8246d05fa168_12,4d85652b-e60d-4510-abed-8246d05fa168_11,4d85652b-e60d-4510-abed-8246d05fa168_26,4d85652b-e60d-4510-abed-8246d05fa168_23,4d85652b-e60d-4510-abed-8246d05fa168_17,...,4d85652b-e60d-4510-abed-8246d05fa168_9,4d85652b-e60d-4510-abed-8246d05fa168_14,4d85652b-e60d-4510-abed-8246d05fa168_16,4d85652b-e60d-4510-abed-8246d05fa168_18,4d85652b-e60d-4510-abed-8246d05fa168_15,4d85652b-e60d-4510-abed-8246d05fa168_22,4d85652b-e60d-4510-abed-8246d05fa168_20,4d85652b-e60d-4510-abed-8246d05fa168_19,4d85652b-e60d-4510-abed-8246d05fa168_24,4d85652b-e60d-4510-abed-8246d05fa168_21
root_mean_squared_log_error,[0.2079557048025764],[0.001786973013283344],[0.30225518015642167],[1.7680386412699534e-05],[0.8784824532540254],[0.000516298968248563],[0.009090548237308779],[0.11374623326227991],[0.08104292067197161],[0.0005642251553285386],...,[0.017214338880493802],[0.009804409289858226],[0.00024071746534826133],[0.06677051829219316],[0.0005242578975496551],[0.036680512687675325],[0.10383890130026459],[0.03999630908570998],[0.13827272940343605],[0.38526657130863295]
explained_variance,[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],...,[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0]
normalized_median_absolute_error,[0.045204041573806863],[0.000529070260659238],[0.06561041351653409],[4.276184211675838e-06],[0.2939919143283009],[0.00012374572788181892],[0.0018129825361777978],[0.027746732992936406],[0.02064001478258373],[0.00013999331698355454],...,[0.0033274033730144163],[0.001979044319190415],[6.33839750012797e-05],[0.01238966924827486],[9.019590243495753e-05],[0.008342920693357866],[0.02354961541246857],[0.009276481201469665],[0.02786251596028677],[0.09272827976047719]
median_absolute_error,[392.9135293595293],[4.598678705650097],[570.2857142857143],[0.03716859316788638],[2555.3777193415913],[1.07559786674877],[15.758444204457419],[241.17460317460322],[179.4030084902178],[1.216821911221056],...,[28.921790118241308],[17.201853222403088],[0.5509335107111232],[107.69100510600508],[0.7839827839646508],[72.51666666666658],[204.6932571651768],[80.63117460317433],[242.18098872681261],[805.9942076780677]
normalized_mean_absolute_error,[0.045204041573806863],[0.000529070260659238],[0.06561041351653409],[4.276184211675838e-06],[0.2939919143283009],[0.00012374572788181892],[0.0018129825361777978],[0.027746732992936406],[0.02064001478258373],[0.00013999331698355454],...,[0.0033274033730144163],[0.001979044319190415],[6.33839750012797e-05],[0.01238966924827486],[9.019590243495753e-05],[0.008342920693357866],[0.02354961541246857],[0.009276481201469665],[0.02786251596028677],[0.09272827976047719]
r2_score,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
normalized_root_mean_squared_error,[0.045204041573806863],[0.000529070260659238],[0.06561041351653409],[4.276184211675838e-06],[0.2939919143283009],[0.00012374572788181892],[0.0018129825361777978],[0.027746732992936406],[0.02064001478258373],[0.00013999331698355454],...,[0.0033274033730144163],[0.001979044319190415],[6.33839750012797e-05],[0.01238966924827486],[9.019590243495753e-05],[0.008342920693357866],[0.02354961541246857],[0.009276481201469665],[0.02786251596028677],[0.09272827976047719]
spearman_correlation,[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],...,[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0]
mean_absolute_percentage_error,[23.216552658752136],[0.179136557516656],[25.712315091976155],[0.0017689130177037324],[150.98783460495096],[0.051649481615863684],[0.9149155256295919],[11.227195954072172],[7.668370426324709],[0.0564328736931512],...,[1.7430326913281748],[0.9870780981972899],[0.024085343761376565],[6.977242993920168],[0.052438233

### Retrieve the Best Model

In [18]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/4d85652b-e60d-4510-abed-8246d05fa168/model_data, 1 files out of an estimated total of 1


In [20]:
!pip install azureml.automl.runtime

     |████████████████████████████████| 2.1 MB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 228 kB 71.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 64.7 MB/s eta 0:00:01
     |████████████████████████████████| 8.7 MB 32.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 68.0 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 58.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 45.7 MB/s eta 0:00:01    |█▋                              | 102 kB 45.7 MB/s eta 0:00:01
     |████████████████████████████████| 803 kB 50.8 MB/s eta 0:00:01
ERROR: pyldavis 3.3.1 requires sklearn, which is not installed.
ERROR: pandas-ml 0.6.1 requires enum34, which is not installed.
ERROR: raiwidgets 0.16.0 has requirement ipython==7.16.1, but you'll have ipython 7.16.3 which is incompatible.
ERROR: raiwidgets 0.16.0 has requirement jinja2==2.11.3, but you'll have jinja2 2.11.2 which is incompatible.
ERROR: pyldavis 3.3.1 has requir

In [19]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

ModuleNotFoundError: No module named 'azureml.automl.runtime'

In [ ]:
best_model.steps

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [21]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bikesharing Train", description="Training bikesharing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bikesharing Train,f14de75d-d5d9-4dfc-aec1-dbf500ee306e,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [22]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [23]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-bike-rest-endpoint"}
                        )

In [24]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  2e370cde-d73c-469d-8d8f-6560fec67137


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [25]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-bike-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…